DSCI100 Final Project - Group 22

# Can Age and Experience Level Predict Hours Played in the Players Dataset?

By Vanessa Mah 76858182, NAME, NAME & NAME


## (1) Introduction 

Qt. provide some relevant background information on the topic so that someone unfamiliar with it will be prepared to understand the rest of your report

- A research group in Computer Science at UBC led by Frank Wood, is collecting data about how people play video games. They have set up a MineCraft server (called PLAICraft), and players' actions are recorded as they navigate through the world. However, running this project is not simple, as they need to target their recruitment efforts and ensure they have enough resources to handle the number of players they attract.

The broad question to be answered is: **Which "kinds" of players are most likely to contribute a large amount of dataso that we can target those players in our recruiting efforts?**

To refine this question, our specific question is: **Can age and experience predict hours played in the Players dataset?**

Qt. identify and fully describe the dataset that was used to answer the question



## (2) Methods & Results 

## (3) Discussion

## (4) References